# cut cross entropy

author: [xiaodongguaAIGC](https://github.com/dhcode-cpp)

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
torch.manual_seed(42)   

## Cross-Entropy

In [2]:
X = torch.tensor([1.0, 3.0, -1.2, 1.1, -0.5, -0.8]) # logits
Y = torch.tensor([0.4, 0.2, 0.1,  0.1, 0.05, 0.15])

### Cross Entropy Pytorch

$CE(X,Y) = \sum_i -\log(\text{softmax}(X))$

In [3]:
loss = nn.CrossEntropyLoss()
l = loss(X,Y)
print(l)

tensor(2.4569)

### Cross Entropy with log softmax

when we calculate cross entropy, we need logprob

In [4]:
l = -Y * F.log_softmax(X, dim = 0 )
l = l.sum()
print(l)

tensor(2.4569)

In [5]:
p = F.softmax(X, dim = 0 )
l = -Y * p.log()
l = l.sum()
print(l)

tensor(2.4569)

## Softmax

### forward

In [6]:
p = torch.exp(X) / (torch.exp(X)).sum()
print(p)
l = - Y * p.log()
l = l.sum()
print(l)

tensor([0.1001, 0.7394, 0.0111, 0.1106, 0.0223, 0.0165])

tensor(2.4569)

### gradient

In [7]:
N = 6 
dX = (p - Y) / 6
print(dX)

tensor([-0.0500,  0.0899, -0.0148,  0.0018, -0.0046, -0.0222])

In [8]:
dpdX = torch.diag(p) - torch.outer(p, p)
dX = -(Y / p) @ dpdX   # first term is (- Y log p)' = (-Y/p)
print(dX)

tensor([-0.2999,  0.5394, -0.0889,  0.0106, -0.0277, -0.1335])

## Log Softmax

### forward

In [9]:
LSE = torch.log( torch.sum( torch.exp( X ) ) )
p = X - LSE # X - LSE
print(p)
l = - Y * p # delete log
l = l.sum()
print(l)

tensor([-2.3019, -0.3019, -4.5019, -2.2019, -3.8019, -4.1019])

tensor(2.4569)

### gradient

首先，L = -∑y_i * log(softmax(x_i))

= -∑y_i * p_i (因为p_i = LogSoftmax(x_i))

求导链式法则：

∂L/∂x_k = ∑_i (∂L/∂p_i * ∂p_i/∂x_k)

其中：

∂L/∂p_i = -y_i

∂p_i/∂x_k = δ_{ik} - softmax(x_k) 

代入得到：

∂L/∂x_k = ∑_i (-y_i * (δ_{ik} - softmax(x_k)))

= -y_k + ∑_i y_i * softmax(x_k)

= -y_k + softmax(x_k) * ∑_i y_i

= softmax(x_k) - y_k (因为∑y_i = 1)


In [10]:
l = - Y * p / N
# p.exp() is softmax result probability
dX = p.exp() - Y # lse is easy revearse p 
print(dX)

tensor([-0.2999,  0.5394, -0.0889,  0.0106, -0.0277, -0.1335])

## Cut Cross Entropy

think about llm model has a big vocabulary, so the classifer project d-dimension to |V|-dimension, and |V| usually is big like 128,000 or 256,000.

classifier proj is big GEMM when the model params < 2B

In [11]:
V = 128
D = 16
BS = 8
block_size = 4
n_b = BS // block_size
d_b = D // block_size
v_b = V // block_size

print(n_b)
print(d_b)
print(v_b)

2

4

32

In [12]:
C = torch.randn(D, V)
E = torch.randn(BS, D)
Y_raw = torch.randn(BS, V)
Y_t = Y_raw / 0.0001 # be sharp
Y = F.softmax(Y_t, dim = 1)
# # print(YY)
# _, label = torch.max(Y, dim = 1)
# print(label)
print(Y.shape) # label

torch.Size([8, 128])

### standard CE

In [13]:
# ce
logits = E @ C 
print(logits.shape)
print(Y.shape)
l = loss(logits, Y)
print(l)

torch.Size([8, 128])

torch.Size([8, 128])

tensor(12.3959)

In [14]:
# softmax -> ce
P = F.softmax(logits, dim = 1)
print(P)
l = -Y * P.log()
print(l.sum() / (BS*V))

tensor([[2.1829e-02, 6.1995e-06, 4.3568e-04,  ..., 6.7271e-05, 5.7252e-06,
         4.6990e-05],
        [4.9314e-03, 6.7518e-06, 2.8711e-07,  ..., 3.5461e-07, 1.6502e-03,
         5.9683e-06],
        [2.3456e-05, 3.4222e-04, 4.7783e-02,  ..., 1.5149e-07, 4.6281e-05,
         3.3086e-05],
        ...,
        [1.8751e-03, 1.0904e-07, 1.7841e-04,  ..., 1.5724e-05, 1.1881e-04,
         5.2249e-04],
        [1.7716e-10, 2.9223e-05, 3.3995e-07,  ..., 3.2832e-06, 9.3845e-07,
         1.8057e-07],
        [2.2445e-04, 9.8148e-06, 1.2590e-05,  ..., 6.5709e-05, 3.8583e-05,
         6.3119e-04]])

tensor(0.0968)

In [15]:
# logsoftmax -> ce
P = F.log_softmax(logits, dim = 1)
l = -Y * P
print(l.sum() / (BS*V))

tensor(0.0968)

### Cut Cross Entropy 

#### *Algorithm 1: forward GEMM*

our target is get  


logprob = X @ C - log \sum_j X[i,:] C[:,j] # row sub

the X[i,1] C[1,j] + X[i,2] C[2,j] + X[i,d] C[d,j] + 

output = -Y * logprob

In [16]:
X = E @ C

In [17]:
print(X.shape)

torch.Size([8, 128])

In [18]:
X_blocks = torch.split(X, 64, dim = 1)
print(X_blocks[0].shape)
print(len(X_blocks))

torch.Size([8, 64])

2

In [19]:
E_blocks = torch.split(E, n_b, dim = 0)
C_blocks = torch.split(C, v_b, dim = 1)

print(len(E_blocks), len(C_blocks))
print(E_blocks[0].shape)
print(C_blocks[0].shape)

4 4

torch.Size([2, 16])

torch.Size([16, 32])

In [20]:
stand_mm = E @ C
print(stand_mm[:5,:5])

tensor([[ 6.2235, -1.9430,  2.3094, -0.8064,  0.9664],
        [ 9.5531,  2.9595, -0.1982, -1.5044, -2.8647],
        [-1.5816,  1.0988,  6.0377,  1.3787,  2.7961],
        [ 3.4435, -4.2511, -3.4568,  4.3576, -2.2228],
        [ 2.9545, -5.2834, -3.8898, -3.6405, -6.7475]])

In [21]:
# CCE : Algorithm 1

O = torch.zeros(BS, V)
O_blocks = torch.split(O, n_b, dim = 0)
O_result = []

for e, o in zip(E_blocks, O_blocks): # i
    e_d_blocks = torch.split(e, d_b, dim = 1)
    c_blocks = torch.split(C, d_b, dim = 0)
    for ed, cd in zip(e_d_blocks, c_blocks): # 
        a = ed @ cd
        o = o + a
    O_result.append(o)

O_mat = torch.cat(O_result, dim = 0)
print(O_mat.shape)

print(O_mat[:5,:5])

torch.Size([8, 128])

tensor([[ 6.2235, -1.9430,  2.3094, -0.8064,  0.9664],
        [ 9.5531,  2.9595, -0.1982, -1.5044, -2.8647],
        [-1.5816,  1.0988,  6.0377,  1.3787,  2.7961],
        [ 3.4435, -4.2511, -3.4568,  4.3576, -2.2228],
        [ 2.9545, -5.2834, -3.8898, -3.6405, -6.7475]])

#### *Algorithm 2: forward logsoftmax*

Original LSE

In [22]:
A = E @ C 
LSE = torch.log( torch.sum( torch.exp(A), dim = 1, keepdim=True ) )
print(LSE)

tensor([[10.0480],
        [14.8652],
        [ 9.0788],
        [11.3516],
        [11.6501],
        [11.5272],
        [12.2498],
        [10.3057]])

In [23]:
# CCE : Algorithm 2

LSE = torch.ones(BS, 1) * -100000.0 # -inf

# LSE part
for n, e in enumerate(E_blocks):
    for v, c in enumerate(C_blocks):
        A = torch.zeros(n_b, v_b)
        e_d_blocks = torch.split(e, d_b, dim = 1)
        c_d_blocks = torch.split(c, d_b, dim = 0)
        for ed, cd in zip(e_d_blocks, c_d_blocks):
            A += ed @ cd
        LSE_nv = torch.log( torch.sum( torch.exp(A), dim = 1, keepdim=True ) )
        LSE[n_b * n : n_b * (n+1)] = torch.log( torch.exp(LSE[n_b * n : n_b * (n+1)]) 
                                                    + torch.exp(LSE_nv))
print(LSE.shape)
print(LSE)

torch.Size([8, 1])

tensor([[10.0480],
        [14.8652],
        [ 9.0788],
        [11.3516],
        [11.6501],
        [11.5272],
        [12.2498],
        [10.3057]])

#### Backward

#### standard backward

In [24]:
# basic backward
# P = softmax(E @ C, dim = 1)
# loss = - Y * torch.log(P)

Y = torch.randn(BS, V)
Y = torch.softmax(Y, dim = 1)

X = E @ C
P = torch.softmax(X, dim = 1)

loss = (- Y * torch.log(P)).mean()
print(loss)

tensor(0.0914)

In [25]:
# Stand backward
N = BS * V

dP = (P - Y) / N
dPdX = torch.ones(V, V)
for i in range(BS):
    dPdX += torch.diag(P[i,:]) - torch.outer(P[i,:], P[i,:]) 

dX = dP @ dPdX 
print(dX.shape)

dE = dP @ C.t()
dC = E.t() @ dP 

print(dE[:5,:5])
print(dC[:5,:5])


torch.Size([8, 128])

tensor([[-2.0505e-03,  5.9219e-05, -3.9728e-04,  1.7721e-03, -1.2872e-03],
        [ 1.4056e-03, -8.0426e-04, -1.4729e-03,  7.2635e-05,  1.3984e-03],
        [ 2.1512e-04, -6.5967e-05, -4.0976e-05, -1.2592e-04, -9.8514e-04],
        [-2.1033e-03,  7.8928e-04, -8.4670e-04, -6.1922e-04, -1.3060e-03],
        [-1.0860e-03, -1.2127e-03,  1.9582e-05,  9.4656e-04,  1.4197e-04]])

tensor([[-6.0297e-06, -9.2597e-06,  2.6474e-05,  1.7252e-05,  2.3637e-05],
        [ 9.8354e-07, -8.1001e-06,  2.7248e-05, -2.1423e-05,  6.6895e-08],
        [ 9.2884e-06,  3.9093e-05, -5.3355e-05,  3.1693e-05, -9.0323e-06],
        [ 3.2662e-05,  2.1504e-05, -8.4147e-05, -5.0974e-05, -1.0336e-05],
        [-1.8036e-05, -2.4595e-05, -2.3332e-05,  3.7324e-06, -3.0484e-06]])

#### LSE Backward

In [26]:
X = E @ C
LSE = torch.log(torch.sum( torch.exp(X), dim = 1, keepdim = True))
print(LSE)
P = X - LSE # 逐行减去LSE
loss = (- Y * P) # (-Y * X + Y * LSE)
print(loss.mean() )


tensor([[10.0480],
        [14.8652],
        [ 9.0788],
        [11.3516],
        [11.6501],
        [11.5272],
        [12.2498],
        [10.3057]])

tensor(0.0914)

In [27]:
dY_dX = -Y / N # P = "X" - LSE
dY_dLSE = torch.sum(Y / N, dim = 1, keepdim = True) # P = X - "LSE"
dLSE_dX = F.softmax(X, dim = 1)
dY_dX_LSE = dY_dLSE * dLSE_dX

dY_dX_total =  dY_dX + dY_dX_LSE

dE = dY_dX_total @ C.t()
dC = E.t() @ dY_dX_total 

print(dE[:5,:5])
print(dC[:5,:5])

tensor([[-2.0505e-03,  5.9219e-05, -3.9728e-04,  1.7721e-03, -1.2872e-03],
        [ 1.4056e-03, -8.0426e-04, -1.4729e-03,  7.2635e-05,  1.3984e-03],
        [ 2.1512e-04, -6.5967e-05, -4.0976e-05, -1.2592e-04, -9.8514e-04],
        [-2.1033e-03,  7.8928e-04, -8.4670e-04, -6.1922e-04, -1.3060e-03],
        [-1.0860e-03, -1.2127e-03,  1.9582e-05,  9.4656e-04,  1.4197e-04]])

tensor([[-6.0297e-06, -9.2597e-06,  2.6474e-05,  1.7252e-05,  2.3637e-05],
        [ 9.8354e-07, -8.1001e-06,  2.7248e-05, -2.1423e-05,  6.6895e-08],
        [ 9.2884e-06,  3.9093e-05, -5.3355e-05,  3.1693e-05, -9.0323e-06],
        [ 3.2662e-05,  2.1504e-05, -8.4147e-05, -5.0974e-05, -1.0336e-05],
        [-1.8036e-05, -2.4595e-05, -2.3332e-05,  3.7324e-06, -3.0484e-06]])

In [28]:
dE_LSE = dLSE_dX @ C.t()
dC_LSE = E.t() @ dLSE_dX 
print(dE_LSE[:5,:5])
print(dC_LSE[:5,:5])

tensor([[-2.1015,  0.0912, -0.4023,  1.6885, -1.3617],
        [ 1.6390, -0.9415, -1.3662,  0.0256,  1.3215],
        [ 0.3928,  0.0740, -0.0271, -0.2499, -1.0956],
        [-2.1898,  0.9732, -0.6191, -0.6700, -1.3252],
        [-0.9124, -1.2397, -0.0917,  0.7983,  0.0990]])

tensor([[-2.5990e-03,  3.7586e-05,  8.3423e-03, -1.6059e-03, -6.1319e-04],
        [ 6.6817e-03, -5.5734e-05, -8.2944e-03, -1.2376e-04, -3.7091e-04],
        [-6.8258e-03, -1.7603e-04, -2.6890e-02, -8.5122e-04, -3.0148e-04],
        [ 3.2146e-02, -2.9805e-04, -4.1456e-02, -3.1811e-05, -1.5265e-03],
        [-3.3457e-02, -2.5087e-04, -4.2576e-02, -1.9649e-03, -2.7716e-04]])

#### Pytorch AutoGrad LSE backward

 仅检验手动计算的梯度是否正确

In [29]:
# # 注意要清除梯度，得到的才准确
# # E.grad.zero_()
# # C.grad.zero_()
# # Y.grad.zero_()

# E.requires_grad = True
# C.requires_grad = True
# Y.requires_grad = True


# X = E @ C
# X.retain_grad()

# LSE = torch.log(torch.sum( torch.exp(X), dim = 1, keepdim = True))
# LSE.retain_grad()

# P = X - LSE # 逐行减去LSE
# P.retain_grad()

# loss = (- Y * P) # (-Y * X + Y * LSE)
# loss.retain_grad()
# # print(loss)

# l = loss.mean()
# l.backward() 

In [30]:
# print(loss.grad[:5,:5])

# print(LSE.grad[:5,:5])
# print(LSE.grad.shape)

# print(X.grad[:5,:5])
# print(E.grad[:5,:5])
# print(C.grad[:5,:5])

#### Algorithm 3: CCE LSE Backward

In [31]:
# CCE : Algorithm 3

is_filter = False
epsilon = 1e-6
# 
dE = torch.zeros_like(E)
dC = torch.zeros_like(C)

dE_blocks = torch.split(dE, n_b, dim = 0)
dC_blocks = torch.split(dC, v_b, dim = 1)

print(Y.shape)
d_LSE = dY_dLSE
print(d_LSE)

# LSE part
for n, e in enumerate(E_blocks):
    for v, c in enumerate(C_blocks):
        
        A = torch.zeros(n_b, v_b)
        e_d_blocks = torch.split(e, d_b, dim = 1)
        c_d_blocks = torch.split(c, d_b, dim = 0)
        for ed, cd in zip(e_d_blocks, c_d_blocks):
            A += ed @ cd # A_nv

        S = torch.exp(A - LSE[n_b * n : n_b * (n+1)]) # softmax

        # filter S
        if is_filter:
            if  (S < epsilon).all():
                print('skip', n, v)
                continue
            
        
        d_LSE_block = d_LSE[n_b * n : n_b * (n+1), :]
        d_index = 0
        for ed, cd in zip(e_d_blocks, c_d_blocks):
            dE_blocks[n][ :, d_index * d_b: (d_index+1)*d_b ] +=  (d_LSE_block * S) @ cd.t()
            dC_blocks[v][ d_index * d_b: (d_index+1)*d_b, : ] +=  ed.t() @ (d_LSE_block * S)
            d_index = d_index + 1


# dLSE gradient part
dE_part_LSE = torch.cat(dE_blocks, dim = 0)
dC_part_LSE = torch.cat(dC_blocks, dim = 1)

print(dE_part_LSE.shape)
print(dC_part_LSE.shape)

print(dE_part_LSE[:5,:5])
print(dC_part_LSE[:5,:5])

torch.Size([8, 128])

tensor([[0.0010],
        [0.0010],
        [0.0010],
        [0.0010],
        [0.0010],
        [0.0010],
        [0.0010],
        [0.0010]])

torch.Size([8, 16])

torch.Size([16, 128])

tensor([[-2.0523e-03,  8.9026e-05, -3.9288e-04,  1.6489e-03, -1.3298e-03],
        [ 1.6006e-03, -9.1940e-04, -1.3342e-03,  2.5002e-05,  1.2905e-03],
        [ 3.8360e-04,  7.2266e-05, -2.6462e-05, -2.4400e-04, -1.0699e-03],
        [-2.1385e-03,  9.5039e-04, -6.0458e-04, -6.5433e-04, -1.2941e-03],
        [-8.9104e-04, -1.2106e-03, -8.9572e-05,  7.7957e-04,  9.6675e-05]])

tensor([[-2.5381e-06,  3.6705e-08,  8.1468e-06, -1.5683e-06, -5.9882e-07],
        [ 6.5251e-06, -5.4428e-08, -8.1000e-06, -1.2086e-07, -3.6222e-07],
        [-6.6658e-06, -1.7190e-07, -2.6260e-05, -8.3127e-07, -2.9442e-07],
        [ 3.1392e-05, -2.9107e-07, -4.0484e-05, -3.1066e-08, -1.4908e-06],
        [-3.2672e-05, -2.4499e-07, -4.1578e-05, -1.9189e-06, -2.7066e-07]])

In [32]:
# dX part dE, dC
dE_part_X = dY_dX @ C.t()
dC_part_X = E.t() @ dY_dX

# dX Total
dE_total = dE_part_X + dE_part_LSE
dC_total = dC_part_X + dC_part_LSE

print(dE_total[:5, :5])
print(dC_total[:5, :5])

tensor([[-2.0505e-03,  5.9219e-05, -3.9728e-04,  1.7721e-03, -1.2872e-03],
        [ 1.4056e-03, -8.0426e-04, -1.4729e-03,  7.2635e-05,  1.3984e-03],
        [ 2.1512e-04, -6.5967e-05, -4.0976e-05, -1.2592e-04, -9.8514e-04],
        [-2.1033e-03,  7.8929e-04, -8.4670e-04, -6.1922e-04, -1.3060e-03],
        [-1.0860e-03, -1.2127e-03,  1.9582e-05,  9.4656e-04,  1.4197e-04]])

tensor([[-6.0297e-06, -9.2597e-06,  2.6474e-05,  1.7252e-05,  2.3637e-05],
        [ 9.8354e-07, -8.1001e-06,  2.7248e-05, -2.1423e-05,  6.6895e-08],
        [ 9.2884e-06,  3.9093e-05, -5.3355e-05,  3.1693e-05, -9.0323e-06],
        [ 3.2662e-05,  2.1504e-05, -8.4147e-05, -5.0974e-05, -1.0336e-05],
        [-1.8036e-05, -2.4595e-05, -2.3332e-05,  3.7324e-06, -3.0484e-06]])

#### Algorithm 4: CCE Backward total

In [33]:
# print(Y)
# Y_t = Y / 0.001 # be sharp
# YY = F.softmax(Y_t, dim = 1)
# print(YY)
_, label = torch.max(Y, dim = 1)
print(label)

print(Y[0, label[0]])

tensor([  7,  22,  36, 103,  74,  92, 124,  37])

tensor(0.0611)

##### max likelihood version CE

In [34]:
print(P.shape)

torch.Size([8, 128])

In [35]:
print(label.unsqueeze(0))
P_CEL = torch.gather(P, dim = 1, index = label.unsqueeze(1))
loss = -P_CEL
l = loss.mean()
print(loss)
print(l)

tensor([[  7,  22,  36, 103,  74,  92, 124,  37]])

tensor([[ 4.2771],
        [19.7241],
        [ 9.0286],
        [14.6713],
        [12.1181],
        [19.8909],
        [18.0299],
        [11.2865]])

tensor(13.6283)

In [36]:
loss = nn.CrossEntropyLoss()
l_torch = loss(X, label)
print(l_torch)

tensor(13.6283)

In [37]:
print(d_LSE)
print(l_torch/BS)

tensor([[0.0010],
        [0.0010],
        [0.0010],
        [0.0010],
        [0.0010],
        [0.0010],
        [0.0010],
        [0.0010]])

tensor(1.7035)

In [53]:
# CCE : Algorithm 4

# M = torch.zeros(BS, V)
# for i in range(BS):
#     M[i, label[i]] = 1
M_blocks = torch.split(Y, n_b, dim = 0)

dE = torch.zeros_like(E)
dC = torch.zeros_like(C)

dE_blocks = torch.split(dE, n_b, dim = 0)
dC_blocks = torch.split(dC, v_b, dim = 1)

d_LCE = torch.ones(BS, 1) / (BS*V)


for n, e in enumerate(E_blocks):
    
    M_v_blcoks = torch.split(M_blocks[n], v_b, dim = 1)
                                 
    for v, c in enumerate(C_blocks):
        
        A = torch.zeros(n_b, v_b)
        e_d_blocks = torch.split(e, d_b, dim = 1)
        c_d_blocks = torch.split(c, d_b, dim = 0)
        for ed, cd in zip(e_d_blocks, c_d_blocks):
            A += ed @ cd # A_nv

        S = torch.exp(A - LSE[n_b * n : n_b * (n+1)]) 
        G = -M_v_blcoks[v] + S                          # Total Gradient
                
        # filter S
        if is_filter:
            if  (G.abs() < epsilon).all():
                print('skip',n,v)
                continue
        
        d_LCE_block = d_LCE[n_b * n : n_b * (n+1), :]
        d_index = 0
        for ed, cd in zip(e_d_blocks, c_d_blocks):
            dE_blocks[n][ :, d_index * d_b: (d_index+1)*d_b ] +=  (d_LCE_block * G) @ cd.t()
            dC_blocks[v][ d_index * d_b: (d_index+1)*d_b, : ] +=  ed.t() @ (d_LCE_block * G)
            d_index = d_index + 1


# dLSE gradient part
dE = torch.cat(dE_blocks, dim = 0)
dC = torch.cat(dC_blocks, dim = 1)

print(dE[:5,:5])
print(dC[:5,:5])

tensor([[-2.0505e-03,  5.9219e-05, -3.9728e-04,  1.7721e-03, -1.2872e-03],
        [ 1.4056e-03, -8.0426e-04, -1.4729e-03,  7.2635e-05,  1.3984e-03],
        [ 2.1512e-04, -6.5967e-05, -4.0976e-05, -1.2592e-04, -9.8514e-04],
        [-2.1033e-03,  7.8929e-04, -8.4670e-04, -6.1922e-04, -1.3060e-03],
        [-1.0860e-03, -1.2127e-03,  1.9582e-05,  9.4656e-04,  1.4197e-04]])

tensor([[-6.0297e-06, -9.2597e-06,  2.6474e-05,  1.7252e-05,  2.3637e-05],
        [ 9.8354e-07, -8.1001e-06,  2.7248e-05, -2.1423e-05,  6.6895e-08],
        [ 9.2884e-06,  3.9093e-05, -5.3355e-05,  3.1693e-05, -9.0323e-06],
        [ 3.2662e-05,  2.1504e-05, -8.4147e-05, -5.0974e-05, -1.0336e-05],
        [-1.8036e-05, -2.4595e-05, -2.3332e-05,  3.7324e-06, -3.0484e-06]])